# Gradio Interface to Chat Description
This is the final product showcasing the RevoCRM built in the Gradio environment. The current version uses DummyCRM, a test module that simulates the interface and functionality for demonstration purposes.

Our RevoCRM can:

* Generate intelligent business insights and pipeline analyses using advanced prompts, providing evaluations of conversion rates, deal performance, and strategic recommendation for sales optimization (TO-DO).

* Produce detailed Matplotlib/Seaborn charts visualizing opportunity pipeline stages, revenue distribution by industry, lead conversion rates, and deal amount distributions across the sales organization.

Example queries it can handle are:
- 'Give me a summary of [account name]'
- 'Draft a follow-up email for [account name]'
- Just type any question to see what it can help with!

## Step 1: Import Packages

In [1]:
import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import time
import google.generativeai as genai
from dotenv import load_dotenv

/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


## Step 2: Setup for Gemini and API Key

In [2]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [3]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')

Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'mo

## Step 3: Load Data

In [4]:
# Load clean data
base_path = "data_directory/clean_data"

# Read CSV files
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

## Step 4: Dashboard Visualization Functions
These functions create the 4 key CRM dashboard charts:
1. Pipeline Value by Deal Stage - Shows total deal value across pipeline stages
2. Revenue by Industry - Displays revenue from Closed Won deals by sector
3. Lead Conversion Rate - Compares converted vs non-converted leads
4. Deal Amount Distribution - Shows distribution of deal sizes

In [5]:
# This function applies account, agent, and timeframe filters to pipeline data
# It's used by all visualization functions to ensure consistent filtering

def apply_filters(df, account=None, agent=None, timeframe=None):
    """
    Apply filters to pipeline DataFrame
    
    Args:
        df: Pipeline DataFrame
        account: Account name filter (optional)
        agent: Sales agent name filter (optional)
        timeframe: Minimum date filter in YYYY-MM-DD format (optional)
    
    Returns:
        Filtered DataFrame
    """
    data = df.copy()
    
    # Filter by account if specified
    if account:
        data = data[data["account"] == account]
    
    # Filter by sales agent if specified
    if agent:
        data = data[data["sales_agent"] == agent]
    
    # Filter by timeframe (engagement date) if specified
    if timeframe:
        try:
            data["engage_date"] = pd.to_datetime(data["engage_date"])
            data = data[data["engage_date"] >= pd.to_datetime(timeframe)]
        except:
            print("Invalid date format. Use YYYY-MM-DD.")
    
    return data

### Chart 1: Pipeline Value by Deal Stage
Makes a bar chart showing total deal value grouped by pipeline stage and helps identify which stages contain the most value.

In [6]:
def plot_pipeline_by_stage(pipeline_df):
    """
    Create bar chart of pipeline value by deal stage
    
    Args:
        pipeline_df: Filtered pipeline DataFrame
    
    Returns:
        Matplotlib figure object
    """
    # Handle empty data case
    if pipeline_df.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No data available for selected filters', 
                ha='center', va='center', fontsize=12)
        ax.set_title("Pipeline Value by Deal Stage")
        return fig
    
    # Group by deal stage and sum values
    summary = pipeline_df.groupby("deal_stage")["close_value"].sum().sort_values()
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    summary.plot(kind="bar", ax=ax, color='steelblue')
    ax.set_title("Pipeline Value by Deal Stage", fontsize=14, fontweight='bold')
    ax.set_xlabel("Deal Stage", fontsize=12)
    ax.set_ylabel("Total Deal Value ($)", fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    return fig

### Chart 2: Revenue by Industry/Sector
Shows revenue distribution from Closed Won deals only and helps identify most profitable industry sectors.

In [7]:
def plot_revenue_by_industry(pipeline_df, accounts_df):
    """
    Create bar chart of revenue by industry (Closed Won deals only)
    
    Args:
        pipeline_df: Filtered pipeline DataFrame
        accounts_df: Accounts DataFrame with sector information
    
    Returns:
        Matplotlib figure object
    """
    # Handle empty data case
    if pipeline_df.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No data available for selected filters', 
                ha='center', va='center', fontsize=12)
        ax.set_title("Revenue by Industry (Closed Won Only)")
        return fig

    # Merge pipeline with account data to get sector information
    merged = pipeline_df.merge(accounts_df, on="account", how="left")
    
    # Filter for Closed Won deals only (real revenue)
    won = merged[merged["deal_stage"].str.contains("Won", case=False, na=False)]
    
    # Handle case where no Won deals exist
    if won.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No Closed Won deals in selected data', 
                ha='center', va='center', fontsize=12)
        ax.set_title("Revenue by Industry (Closed Won Only)")
        return fig

    # Group by sector and sum revenue
    revenue = won.groupby("sector")["close_value"].sum().sort_values()

    # Create bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    revenue.plot(kind="bar", ax=ax, color='seagreen')
    ax.set_title("Revenue by Industry (Closed Won Only)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Industry Sector", fontsize=12)
    ax.set_ylabel("Revenue ($)", fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    return fig

### Chart 3: Lead Conversion Rate
Makes a bar chart comparing converted vs non-converted leads and a lead is "converted" if deal stage contains "Won".

In [8]:
def plot_conversion_rate(pipeline_df):
    """
    Create bar chart showing converted vs non-converted leads
    
    Args:
        pipeline_df: Filtered pipeline DataFrame
    
    Returns:
        Matplotlib figure object
    """
    # Handle empty data case
    if pipeline_df.empty:
        fig, ax = plt.subplots(figsize=(7, 6))
        ax.text(0.5, 0.5, 'No data available for selected filters', 
                ha='center', va='center', fontsize=12)
        ax.set_title("Lead Conversion Count")
        return fig
    
    # Create conversion flag (True if deal stage contains "Won")
    data = pipeline_df.copy()
    data["converted"] = data["deal_stage"].str.contains("Won", case=False, na=False)
    
    # Count converted vs non-converted
    conv_counts = data["converted"].value_counts()

    labels = ["Converted", "Not Converted"]
    values = [conv_counts.get(True, 0), conv_counts.get(False, 0)]

    # Create bar chart
    fig, ax = plt.subplots(figsize=(7, 6))
    colors = ['#2ecc71', '#e74c3c']  # Green for converted, red for not converted
    ax.bar(labels, values, color=colors)
    ax.set_title("Lead Conversion Count", fontsize=14, fontweight='bold')
    ax.set_ylabel("Number of Leads", fontsize=12)
    
    # Add value labels on bars
    for i, v in enumerate(values):
        ax.text(i, v + max(values)*0.02, str(v), ha='center', fontweight='bold')
    
    plt.tight_layout()
    return fig

### Chart 4: Deal Amount Distribution
Makes a histogram showing distribution of deal sizes and helps understand typical deal values and identify outliers

In [9]:
def plot_deal_amount_distribution(pipeline_df):
    """
    Create histogram of deal amount distribution
    
    Args:
        pipeline_df: Filtered pipeline DataFrame
    
    Returns:
        Matplotlib figure object
    """
    # Handle empty data case
    if pipeline_df.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.text(0.5, 0.5, 'No data available for selected filters', 
                ha='center', va='center', fontsize=12)
        ax.set_title("Deal Amount Distribution")
        return fig
    
    # Create histogram
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(pipeline_df["close_value"], bins=20, edgecolor='black', color='coral', alpha=0.7)
    ax.set_title("Deal Amount Distribution", fontsize=14, fontweight='bold')
    ax.set_xlabel("Deal Amount ($)", fontsize=12)
    ax.set_ylabel("Count", fontsize=12)
    
    # Add grid for better readability
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    return fig

### Unified Chart Generator
This function routes to the appropriate visualization based on user selection


In [10]:
def generate_chart(account_filter, agent_filter, timeframe_filter, chart_type="Pipeline by Stage"):
    """
    Generate selected chart type with applied filters
    
    Args:
        account_filter: Account name to filter by (or None)
        agent_filter: Sales agent to filter by (or None)
        timeframe_filter: Minimum date to filter by (or None)
        chart_type: Type of chart to generate
    
    Returns:
        Matplotlib figure object
    """
    # Apply filters to pipeline data
    filtered = apply_filters(pipeline, account_filter, agent_filter, timeframe_filter)
    
    # Route to appropriate visualization function
    if chart_type == "Pipeline by Stage":
        return plot_pipeline_by_stage(filtered)
    elif chart_type == "Revenue by Industry":
        return plot_revenue_by_industry(filtered, accounts)
    elif chart_type == "Lead Conversion":
        return plot_conversion_rate(filtered)
    elif chart_type == "Deal Distribution":
        return plot_deal_amount_distribution(filtered)
    else:
        # Default to pipeline by stage if unknown chart type
        return plot_pipeline_by_stage(filtered)

## Step 5: Integrate AI Agents and Search Modules

In [11]:
from agents import EmailDraftingAgent, AccountSummaryAgent, InsightGenerationAgent

### CRM and Search Modules

In [12]:
class CRM:
    """
    CRM class that routes queries to appropriate agents.
    """
    
    def __init__(self, accounts_df, pipeline_df, teams_df, products_df):
        self.accounts = accounts_df
        self.pipeline = pipeline_df
        self.teams = teams_df
        self.products = products_df
        
        # Initialize agents
        print("Initializing AI agents...")
        self.account_agent = AccountSummaryAgent()
        self.email_agent = EmailDraftingAgent()
        print("AI agents ready")
    
    def process_query(self, query, context=None):
        """
        Process user queries and route to appropriate agent.
        """
        query_lower = query.lower()
        
        # Check for account summary requests
        if any(word in query_lower for word in ['summary', 'summarize', 'overview', 'tell me about']):
            return self._handle_account_summary(query, context)
        
        # Check for email drafting requests
        elif any(word in query_lower for word in ['email', 'draft', 'write', 'compose', 'send']):
            return self._handle_email_request(query, context)
        
        # Default: provide helpful guidance
        else:
            return self._handle_general_query(query, context)
    
    def _handle_account_summary(self, query, context):
        """Handle account summary requests."""
        try:
            # Try to get account from context
            account_name = None
            
            # First check if 'account' key exists in context
            if context and 'account' in context:
                account_name = context['account']
            
            # If not found, try to extract from query
            if not account_name:
                for acc in self.accounts['account'].unique():
                    if acc.lower() in query.lower():
                        account_name = acc
                        break
            
            if not account_name:
                return "Please specify an account name or select one from the filter.\n\nExample: 'Give me a summary of [account name]'"
            
            print(f"Looking up account: {account_name}")
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found. Please check the spelling or select from the dropdown."
            
            account_info = account_row.iloc[0]
            
            # Prepare account data with safe column access
            account_data = {
                'account_name': str(account_info.get('account', account_name)),
                'sector': str(account_info.get('sector', 'Unknown')),
                'revenue': float(account_info.get('revenue', 0)) if pd.notna(account_info.get('revenue')) else 0,
                'employees': int(account_info.get('employees', 0)) if pd.notna(account_info.get('employees')) else 0,
                'location': str(account_info.get('office_location', 'Unknown'))
            }
            
            print(f"Account data prepared: {account_data['account_name']}")
            
            # Get opportunities for this account
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            if not account_opps.empty:
                won_deals = account_opps[account_opps['deal_stage'].str.lower() == 'won']
                account_data['total_opportunities'] = len(account_opps)
                account_data['won_deals'] = len(won_deals)
                account_data['total_revenue'] = float(won_deals['close_value'].sum())
                print(f"Found {len(account_opps)} opportunities")
            
            # Calculate simple health score
            health_score = 0.75  # Default
            if not account_opps.empty:
                won_count = len(won_deals)
                total_closed = len(account_opps[account_opps['deal_stage'].str.lower().isin(['won', 'lost'])])
                if total_closed > 0:
                    health_score = won_count / total_closed
            
            print(f"Health score: {health_score:.2f}")
            
            # Generate summary using agent
            print(f"Generating AI summary...")
            summary = self.account_agent.create_summary(account_data, health_score)
            
            print(f"Summary generated successfully!")
            return summary
            
        except Exception as e:
            # Show the actual error for debugging
            import traceback
            error_details = traceback.format_exc()
            print(f"ERROR:\n{error_details}")
            return f"Error generating account summary:\n\n{str(e)}\n\nPlease check the console for details."
    
    def _handle_email_request(self, query, context):
        """Handle email drafting requests."""
        try:
            # Determine email type from query
            if 'follow' in query.lower():
                email_type = 'follow_up'
            elif 'intro' in query.lower():
                email_type = 'introduction'
            elif 'proposal' in query.lower():
                email_type = 'proposal'
            elif 'check' in query.lower():
                email_type = 'check_in'
            else:
                email_type = 'follow_up'  # Default - Fixed: underscore
            
            print(f"Email type: {email_type}")
            
            # Get account from context
            account_name = context.get('account') if context else None
            
            if not account_name:
                return "Please select an account from the filter to draft an email."
            
            print(f"🔍 Drafting email for: {account_name}")
            
            # Get account data
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if account_row.empty:
                return f"Account '{account_name}' not found."
            
            account_info = account_row.iloc[0]
            
            account_data = {
                'account_name': str(account_info.get('account', account_name)),
                'sector': str(account_info.get('sector', 'Unknown')),
                'revenue': float(account_info.get('revenue', 0)) if pd.notna(account_info.get('revenue')) else 0,
                'employees': int(account_info.get('employees', 0)) if pd.notna(account_info.get('employees')) else 0
            }
            
            # Get opportunity data if exists
            opportunity_data = None
            account_opps = self.pipeline[self.pipeline['account'] == account_name]
            
            if not account_opps.empty:
                opp = account_opps.iloc[0]
                opportunity_data = {
                    'product': str(opp.get('product', 'Unknown')),
                    'deal_stage': str(opp.get('deal_stage', 'Unknown')),
                    'close_value': float(opp.get('close_value', 0)) if pd.notna(opp.get('close_value')) else 0
                }
                print(f"Opportunity found: {opportunity_data['product']}")
            
            # Determine tone
            tone = 'friendly' if 'friendly' in query.lower() else 'professional'
            
            # Generate email using agent
            print(f"Generating {email_type} email...")
            email = self.email_agent.draft_email(
                email_type=email_type,
                account_data=account_data,
                opportunity_data=opportunity_data,
                tone=tone
            )
            
            print(f"Email generated successfully!")
            return email
            
        except Exception as e:
            # Show the actual error for debugging
            import traceback
            error_details = traceback.format_exc()
            print(f"ERROR:\n{error_details}")
            return f"Error generating email:\n\n{str(e)}\n\nPlease check the console for details."
    
    def _handle_general_query(self, query, context):
        """Handle general queries with helpful guidance."""
        
        # Provide some quick stats
        response = "Sorry! I cannot help with that.\n\n"
        response += "**RevoCRM - What I Can Help With:**\n\n"
        response += "**Account Summaries**: Ask me to summarize any account\n"
        response += "   Example: 'Give me a summary of [account name]'\n\n"
        response += "**Email Drafting**: I can draft sales emails for you\n"
        response += "   Example: 'Draft a follow-up email for [account name]'\n\n"
        response += "**Quick Stats**:\n"
        response += f"   • Total Accounts: {len(self.accounts)}\n"
        response += f"   • Total Opportunities: {len(self.pipeline)}\n"
        response += f"   • Won Deals: {len(self.pipeline[self.pipeline['deal_stage'].str.lower() == 'won'])}\n\n"
        response += "**Tip**: Select an account from the filter and ask me about it!"
        
        return response

In [13]:
# Create the CRM instance
crm = CRM(
    accounts_df=accounts,
    pipeline_df=pipeline,
    teams_df=teams,
    products_df=products
)

Initializing AI agents...
AI agents ready


## Step 6: Chatbot and Dropdown Implementations

### Format Agents Names

In [14]:
# Helper function to format agent names for display
def format_name(name):
    """Convert 'acme_corporation' to 'Acme Corporation'"""
    if name == "None":
        return "None"
    return name.replace("_", " ").title()

def unformat_name(display_name):
    """Convert 'Acme Corporation' back to 'acme_corporation'"""
    if display_name == "None":
        return "None"
    return display_name.replace(" ", "_").lower()

In [15]:
# CRM chatbot function
def crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter, chart_type):
    context = {}
    if account_filter and account_filter != "None":
        row = accounts[accounts["account"] == account_filter].iloc[0].to_dict()
        context.update(row)
    if agent_filter and agent_filter != "None":
        context["sales_agent"] = agent_filter
    if timeframe_filter:
        context["timeframe"] = timeframe_filter

    llm_response = crm.process_query(user_query, context=context)
    
    # Pass chart_type to generate_chart
    chart = generate_chart(
        account_filter if account_filter != "None" else None,
        agent_filter if agent_filter != "None" else None,
        timeframe_filter,
        chart_type
    )
    return llm_response, chart

In [16]:
# wrapper to handle "None" dropdowns
account_choices = ["None"] + sorted([format_name(account) for account in accounts["account"].unique()])
agent_choices = ["None"] + sorted([format_name(agent) for agent in pipeline["sales_agent"].unique()])

def crm_chatbot_wrapper(user_query, account_filter, agent_filter, timeframe_filter, chart_type):
    """
    Wrapper function connecting Gradio interface to CRM chatbot and visualizations
    
    Args:
        user_query: User's natural language query
        account_filter: Selected account filter (or "None")
        agent_filter: Selected sales agent filter (or "None")
        timeframe_filter: Date filter string (or None)
        chart_type: Selected dashboard chart type
    
    Returns:
        CRM Chatbot response and generated chart
    """
    # Convert "None" string to actual None
    if account_filter == "None":
        account_filter = None
    else:
        account_filter = unformat_name(account_filter) # Convert display name back to original name with '_'
    
    if agent_filter == "None":
        agent_filter = None
    else:
        agent_filter = unformat_name(agent_filter) # Convert display name back to original name with '_'
    
    return crm_chatbot(user_query, account_filter, agent_filter, timeframe_filter, chart_type)

## New and Final Gradio Interface

In [ ]:
# Gradio Interface
with gr.Blocks() as demo:
    # Header section
    gr.Markdown("<h1 style='text-align: center; font-size: 40px;'>RevoCRM</h1>")
    gr.Markdown("<h2 style='text-align: center; font-size: 20px;'>Salesforce Team2B</h2>")
    gr.Markdown("<p style='text-align: center; font-size: 15px;'>RevoCRM is your go-to AI assistant for valuable insights. It'll help you get any answer you need!</p>")
    gr.Markdown("<p style='text-align: center; font-size: 16px; font-weight: bold;'>Ask Questions. Get Insights.</p>")

    with gr.Row():
        # LEFT COLUMN: Filters and Chart Selector
        with gr.Column(scale=1):
            gr.Markdown("### Filters")
            
            # Account filter dropdown
            account_dropdown = gr.Dropdown(
                choices=account_choices, 
                label="Account Filter",
                value="None",
                info="Filter by specific account"
            )
            
            # Sales agent filter dropdown
            agent_dropdown = gr.Dropdown(
                choices=agent_choices, 
                label="Sales Agent Filter",
                value="None",
                info="Filter by sales representative"
            )
            
            # Timeframe text input
            timeframe_input = gr.Textbox(
                label="Timeframe (YYYY-MM-DD)", 
                placeholder="e.g., 2016-01-01",
                info="Show data from this date forward"
            )
            
            gr.Markdown("---")  # Visual separator
            gr.Markdown("### Dashboard Chart")
            
            # Chart type selector
            chart_type_dropdown = gr.Dropdown(
                choices=[
                    "Pipeline by Stage",
                    "Revenue by Industry", 
                    "Lead Conversion",
                    "Deal Distribution"
                ],
                label="Visualization Type",
                value="Pipeline by Stage",
                info="Select dashboard chart to display"
            )

            gr.Markdown("---")  # Visual separator
            gr.Markdown("### Chat with CRM Assistant")

            # User query input
            user_input = gr.Textbox(
                label="Ask Anything", 
                placeholder="e.g., 'Give me a summary of an account,' 'Write a friendly intro email', 'Generate an insight on sales performance'",
                lines=2
            )
            
            # Submit button
            submit_btn = gr.Button("Submit", variant="primary")

        # MIDDLE COLUMN: Chat Interface
        with gr.Column(scale=2):
            
            # AI response output
            gr.Markdown("### AI Response")
            llm_output = gr.Textbox(
                label="", 
                lines=10,
                interactive=False
            )
            
            gr.Markdown("### Dashboard Visualization")
            
            # Chart display area
            chart_output = gr.Plot(label="Chart")

    # Connect submit button to wrapper function
    submit_btn.click(
        fn=crm_chatbot_wrapper,
        inputs=[user_input, account_dropdown, agent_dropdown, timeframe_input, chart_type_dropdown],
        outputs=[llm_output, chart_output]
    )

# Launch the interface
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e9d7155aab8f52f391.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Looking up account: bioholding
Account data prepared: bioholding
Found 94 opportunities
Health score: 0.59
Generating AI summary...
Summary generated successfully!
